# Tutorial 2 - Hierarchical Agents

## Functionalities:
- An agent (referred as meta agent) can also assign other agents (referred as inner agents) as the function call, thereby creating a hierarchical structure
- Whenever the inner agent is called, we will run the full inner agent's process until termination
- Maximum number of steps for each agent (meta agent or inner agent) can be defined in `max_subtasks` (default: 5) variable when initialising agent

## Additional Details:
- The inner agent will take in meta agent's instruction as input
- In addition, the inner agent will also have access to meta agent's task and subtasks completed for global context
- Inner agent's output will directly go into meta agent's subtasks completed dictionary, enabling it to directly update global context

## Design Philosophy:
- Bottom layers can get context of the top layers for more context-specific generation
- Top layers choose bottom layers in a hierarchical fashion so as to limit the number of choices at each level for better performance
- `subtasks_completed` serves as the global context for the meta agents and the inner agents, and can be summarised as needed, e.g. when performing a new task to filter out irrelevant information 

# Install TaskGen

In [1]:
# !pip install taskgen-ai

In [2]:
# Set up API key and do the necessary imports
import os
from taskgen import *

os.environ['OPENAI_API_KEY'] = '<YOUR API KEY HERE>'

# Define Agents and Meta Agents
- You can also create a Meta Agent that uses other Agents (referred to as Inner Agents) as functions
- Create your Meta agent using `Agent()` (Note: No different from usual process of creating Agents - your Meta Agent is also an Agent)
- Set up an Inner Agent list and assign it to your Meta agent using `assign_agents(agent_list)`

## Example Meta Agent Setup
```python
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them')
    ]

my_agent.assign_agents(agent_list)
```

## Run the Meta Agent
- Let us run the agent and see the interactions between the Meta Agent and Inner Agents to solve the task!
```python
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')
```

In [3]:
def random_number_from_string(input_string):
    ''' Returns a random number between 1 and 10 when given an input string '''
    import hashlib
    import random
    # Hash the input string
    hashed_string = hashlib.sha256(input_string.encode()).hexdigest()
    
    # Convert the hashed string to an integer
    hashed_integer = int(hashed_string, 16)
    
    # Seed the random number generator with the hashed integer
    random.seed(hashed_integer)
    
    # Generate a random number between 1 and 10
    return random.randint(1, 10)

In [4]:
# assign a dish pricing function for economist
# Note: Try to make the output informative that is understandable by the Agent - use free text or json to explain the output so that someone can understand without the input
def dish_price(list_of_dish_names: list) -> dict:
    ''' Takes in list_of_dish_names and outputs price '''
    if not isinstance(list_of_dish_names, list):
        list_of_food_items = [list_of_dish_names]
    output = {}
    for each in list_of_dish_names:
        output[each] = '$'+str(random_number_from_string(each))
    return output
    
fp = Function(fn_description = "Takes in <list_of_dish_names: list> and outputs price", 
              output_format = {"Price": "Dish name as key and price as value"}, 
              examples = {"list_of_dish_names": ["Pasta", "Pizza"], "Price": [{'Pasta': '$5'}, {'Pizza':'$2'}]},
              external_fn = dish_price)

In [5]:
# Define your meta-agent
my_agent = Agent('Menu Creator', 
                 'Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.')

In [6]:
# Define your agent list. Note you can just assign functions to the agent in place using .assign_functions(function_list)
agent_list = [
    Agent('Chef', 'Takes in dish names and generates ingredients for each of them. Does not generate prices.'),
    Agent('Boss', 'Does final quality check on menu items'),
    Agent('Creative Writer', 'Takes in a cuisine type and generates interesting dish names and descriptions. Does not generate prices or ingredients.', max_subtasks = 2),
    Agent('Economist', 'Takes in dish names and comes up with fictitious pricing for each of them').assign_functions([fp])
    ]

In [7]:
my_agent.assign_agents(agent_list)

# Approach 1: Run a generic task with the Meta-Agent
- Let us run a general task to create 5 menu items for an Italian restaurant
- You can also specify how Agents interact with one another in the task, otherwise the Meta-Agent will infer what should be done

In [8]:
output = my_agent.run('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

Subtask identified: Generate interesting dish names and descriptions using Creative Writer for 5 menu items
Calling function Creative Writer with parameters {'instruction': 'Generate interesting dish names and descriptions for 5 Italian food menu items.'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Brainstorm creative dish names and descriptions for Italian food menu items.
Getting LLM to perform the following task: Brainstorm creative dish names and descriptions for Italian food menu items.
> {'MenuItems': [{'name': 'Ravioli di Mare', 'description': 'Delicate ravioli filled with a blend of fresh seafood, served in a creamy tomato sauce.', 'ingredients': 'Seafood, pasta, tomato sauce, cream', 'price': 'Market Price'}, {'name': 'Pollo alla Griglia', 'description': 'Grilled chicken breast marinated in herbs and olive oil, served with roasted vegetables.', 'ingredients': 'Chicken breast, herbs, olive oil, vegetables', 'price': 'Market Price'}, {'name': 'Tortellini a

In [9]:
output = my_agent.reply_user()

{'MenuItems': [{'name': 'Ravioli di Mare', 'description': 'Delicate ravioli filled with a blend of fresh seafood, served in a creamy tomato sauce.', 'ingredients': 'Seafood, pasta, tomato sauce, cream', 'price': '$2'}, {'name': 'Pollo alla Griglia', 'description': 'Grilled chicken breast marinated in herbs and olive oil, served with roasted vegetables.', 'ingredients': 'Chicken breast, herbs, olive oil, vegetables', 'price': '$8'}, {'name': 'Tortellini al Forno', 'description': 'Baked tortellini stuffed with ricotta and spinach, topped with melted mozzarella cheese.', 'ingredients': 'Tortellini, ricotta, spinach, mozzarella cheese', 'price': '$10'}, {'name': 'Gnocchi Gorgonzola', 'description': 'Soft potato gnocchi in a creamy gorgonzola sauce, sprinkled with toasted walnuts.', 'ingredients': 'Gnocchi, gorgonzola cheese, walnuts, cream', 'price': '$9'}, {'name': 'Tiramisu Classico', 'description': 'Traditional Italian dessert made with layers of coffee-soaked ladyfingers and mascarpone

In [10]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Brainstorm creative dish names and descriptions for Italian food menu items.
{'MenuItems': [{'name': 'Ravioli di Mare', 'description': 'Delicate ravioli filled with a blend of fresh seafood, served in a creamy tomato sauce.', 'ingredients': 'Seafood, pasta, tomato sauce, cream', 'price': 'Market Price'}, {'name': 'Pollo alla Griglia', 'description': 'Grilled chicken breast marinated in herbs and olive oil, served with roasted vegetables.', 'ingredients': 'Chicken breast, herbs, olive oil, vegetables', 'price': 'Market Price'}, {'name': 'Tortellini al Forno', 'description': 'Baked tortellin

# Approach 2: Step through the process yourself as the Meta Agent
- You can use agents manually using `use_agent(agent_name: str, agent_task: str)` The outputs of the agents will be stored into `subtasks_completed` of the meta agent automatically
- You can also interact with the meta agent using `reply_user(task: str, stateful: bool = False)`. This generates a reply for the agent to the `task` using `subtasks_completed` as context. If `stateful` is true, this will also store the output into `subtasks_completed`
- If `subtasks_completed` is too lengthy, you can run `summarise_subtasks_completed(task: str)` This summarises `subtasks_completed` according to what is relevant to the `task`
- If you want to redo a subtask, you can remove the most recent subtask from `subtasks_completed` using `remove_last_subtask()`

In [11]:
# reset the state of the agent
my_agent.reset()
# currently should have no task and nothing in subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: No task assigned
Subtasks Completed: None
Is Task Completed: False


In [12]:
# Assign a task for the meta agent - this helps to contextualise what you give each of the inner agents later on
my_agent.assign_task('Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.')

In [13]:
# This will run the Agent Creative Writer (with shared subtasks_completed as meta agent)
# Output of task will go directly into subtasks_completed - we view the inner agent as simply an extension of the meta agent
my_agent.use_agent('Creative Writer', 'Generate 5 Italian dish names and descriptions')

Calling function Creative Writer with parameters {'instruction': 'Generate 5 Italian dish names and descriptions'}

### Start of Inner Agent: Creative Writer ###
Subtask identified: Research traditional Italian dishes
Getting LLM to perform the following task: Research traditional Italian dishes
> 1. Dish Name: Risotto ai Funghi Description: Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil. 2. Dish Name: Osso Buco alla Milanese Description: Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata. 3. Dish Name: Caprese Salad Description: A classic Italian salad made with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of balsamic glaze. 4. Dish Name: Tiramisu Description: A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder. 5. Dish Name: Spaghetti Carbonara Description: Al dente spaghetti tossed in a creamy sauce made with eggs, pecorino cheese, pancet

In [14]:
# subtasks_completed should be updated with the Creative Agent's outputs
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Research traditional Italian dishes
1. Dish Name: Risotto ai Funghi Description: Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil. 2. Dish Name: Osso Buco alla Milanese Description: Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata. 3. Dish Name: Caprese Salad Description: A classic Italian salad made with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of balsamic glaze. 4. Dish Name: Tiramisu Description: A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and coc

In [15]:
my_agent.use_agent('Chef', 'Generate ingredients for the 5 Italian dishes')

Calling function Chef with parameters {'instruction': 'Generate ingredients for the 5 Italian dishes'}

### Start of Inner Agent: Chef ###
Subtask identified: Research ingredients for Pollo alla Cacciatora
Getting LLM to perform the following task: Research ingredients for Pollo alla Cacciatora
> Pollo alla Cacciatora is a traditional Italian dish made with the following ingredients: 
1. Chicken pieces
2. Tomato sauce
3. Onions
4. Garlic
5. Bell peppers
6. Olive oil
7. Salt
8. Pepper
9. Italian herbs (such as oregano and basil)
10. Red wine (optional)
11. Chicken broth

Subtask identified: Research ingredients for Bruschetta al Pomodoro
Getting LLM to perform the following task: Research ingredients for Bruschetta al Pomodoro
> Bruschetta al Pomodoro is a classic Italian appetizer made with the following ingredients: 
1. Sliced bread (such as baguette or ciabatta)
2. Fresh tomatoes
3. Garlic
4. Fresh basil leaves
5. Extra virgin olive oil
6. Salt
7. Pepper

Subtask identified: Research

In [16]:
my_agent.use_agent('Economist', 'Generate prices for the 5 Italian dishes')

Calling function Economist with parameters {'instruction': 'Generate prices for the 5 Italian dishes'}

### Start of Inner Agent: Economist ###
Subtask identified: Generate prices for Pollo alla Cacciatora
Calling function dish_price with parameters {'list_of_dish_names': ['Pollo alla Cacciatora']}
> {'Price': {'Pollo alla Cacciatora': '$6'}}

Subtask identified: Generate prices for Bruschetta al Pomodoro
Calling function dish_price with parameters {'list_of_dish_names': ['Bruschetta al Pomodoro']}
> {'Price': {'Bruschetta al Pomodoro': '$7'}}

Subtask identified: Generate prices for Gnocchi alla Sorrentina
Calling function dish_price with parameters {'list_of_dish_names': ['Gnocchi alla Sorrentina']}
> {'Price': {'Gnocchi alla Sorrentina': '$10'}}

Subtask identified: Generate prices for Saltimbocca alla Romana
Calling function dish_price with parameters {'list_of_dish_names': ['Saltimbocca alla Romana']}
> {'Price': {'Saltimbocca alla Romana': '$3'}}

Subtask identified: Generate pri

In [17]:
my_agent.use_agent('Boss', 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay')

Calling function Boss with parameters {'instruction': 'Choose only 3 Italian dishes to serve that will please Gordon Ramsay'}

### Start of Inner Agent: Boss ###
Subtask identified: Ensure the dishes are of high quality and authentic
Getting LLM to perform the following task: Ensure the dishes are of high quality and authentic
> As the final quality check agent, I have reviewed the 5 menu items based on Italian food choices. I can confirm that all dishes are of high quality and authentic. The menu items are as follows:

1. Pollo alla Cacciatora
Description: Tender chicken cooked in a savory tomato sauce with onions, garlic, and bell peppers.
Ingredients: Chicken pieces, Tomato sauce, Onions, Garlic, Bell peppers, Olive oil, Salt, Pepper, Italian herbs, Red wine (optional), Chicken broth.
Price: $6

2. Bruschetta al Pomodoro
Description: Toasted bread topped with fresh tomatoes, garlic, basil, and a drizzle of olive oil.
Ingredients: Sliced bread, Fresh tomatoes, Garlic, Fresh basil lea

In [18]:
# Get a targeted response from your agent, and add this to subtask_completed
output = my_agent.reply_user('Choose the 3 menu items for Gordon Ramsay line by line in this format - Dish Name, Dish Description, Dish Ingredients, Dish Price')

Risotto ai Funghi, Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil, Arborio rice, Mixed mushrooms, Truffle oil, Parmesan cheese, Onion, Garlic, Vegetable broth, Butter, White wine, Salt, Black pepper, $12
Osso Buco alla Milanese, Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata, Veal shanks, Tomato, Carrot, Celery, Onion, Garlic, White wine, Beef broth, Gremolata (lemon zest, garlic, parsley), Olive oil, Salt, Black pepper, $18
Tiramisu, A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder, Ladyfingers, Mascarpone cheese, Eggs, Sugar, Coffee, Cocoa powder, Marsala wine, Vanilla extract, $8


In [19]:
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Research traditional Italian dishes
1. Dish Name: Risotto ai Funghi Description: Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil. 2. Dish Name: Osso Buco alla Milanese Description: Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata. 3. Dish Name: Caprese Salad Description: A classic Italian salad made with fresh tomatoes, mozzarella cheese, basil leaves, and a drizzle of balsamic glaze. 4. Dish Name: Tiramisu Description: A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and coc

In [20]:
# you can summarise the subtasks_history for the next task
my_agent.summarise_subtasks_completed('Generate the 3 Italian dishes for Gordon Ramsay in menu format')

Risotto ai Funghi, Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil, Arborio rice, Mixed mushrooms, Truffle oil, Parmesan cheese, Onion, Garlic, Vegetable broth, Butter, White wine, Salt, Black pepper, $12
Osso Buco alla Milanese, Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata, Veal shanks, Tomato, Carrot, Celery, Onion, Garlic, White wine, Beef broth, Gremolata (lemon zest, garlic, parsley), Olive oil, Salt, Black pepper, $18
Tiramisu, A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder, Ladyfingers, Mascarpone cheese, Eggs, Sugar, Coffee, Cocoa powder, Marsala wine, Vanilla extract, $8


In [21]:
# subtask has been summarised, helping to reduce the context significantly for later task generation
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed:
Subtask: Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format
Risotto ai Funghi, Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil, Arborio rice, Mixed mushrooms, Truffle oil, Parmesan cheese, Onion, Garlic, Vegetable broth, Butter, White wine, Salt, Black pepper, $12
Osso Buco alla Milanese, Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata, Veal shanks, Tomato, Carrot, Celery, Onion, Garlic, White wine, Beef broth, Gremolata (lemon zest, garlic, parsley), Olive oil, Salt, Black pepper, $18
Tiramisu, 

In [22]:
# we also allow you to manually remove the last item added to subtasks_completed, so you can revert your agent state if needed for backtracking
my_agent.remove_last_subtask()

Removed last subtask from subtasks_completed: ('Summary of Generate the 3 Italian dishes for Gordon Ramsay in menu format', 'Risotto ai Funghi, Creamy risotto cooked with a variety of mushrooms, finished with a drizzle of truffle oil, Arborio rice, Mixed mushrooms, Truffle oil, Parmesan cheese, Onion, Garlic, Vegetable broth, Butter, White wine, Salt, Black pepper, $12\nOsso Buco alla Milanese, Tender veal shanks braised in a rich tomato and vegetable sauce, served with gremolata, Veal shanks, Tomato, Carrot, Celery, Onion, Garlic, White wine, Beef broth, Gremolata (lemon zest, garlic, parsley), Olive oil, Salt, Black pepper, $18\nTiramisu, A decadent dessert made with layers of coffee-soaked ladyfingers, mascarpone cheese, and cocoa powder, Ladyfingers, Mascarpone cheese, Eggs, Sugar, Coffee, Cocoa powder, Marsala wine, Vanilla extract, $8')


In [23]:
# visualise your agent's status - should have empty subtasks_completed
my_agent.status()

Agent Name: Menu Creator
Agent Description: Creates a menu for a restaurant. Menu item includes Name, Description, Ingredients, Pricing.
Available Functions: ['use_llm', 'end_task', 'Chef', 'Boss', 'Creative Writer', 'Economist']
Task: Generate 5 menu items with name, description, ingredients and price based on Italian food choices. Ensure all parts of menu are generated.
Subtasks Completed: None
Is Task Completed: False
